# 뉴스기업명추출_ver1
2024.07.17

In [1]:
import os
os.chdir('C:/Users/myhyu/OneDrive/Desktop/2024 미래에셋/NewsData/')

In [22]:
# -*- coding: utf-8 -*-

import requests
import pandas as pd

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        response = ""
        with requests.post(self._host + '/testapp/v1/chat-completions/HCX-003',
                           headers=headers, json=completion_request, stream=True) as r:
            for line in r.iter_lines():
                if line:
                    decoded_line = line.decode("utf-8")
                    response += decoded_line + "\n"
        return response

if __name__ == '__main__':
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiY1XHHT1CG2IEYUDfBzRBda5ARQgcLFxGk4gffzMOUc02',
        api_key_primary_val='U8NCSe3jtrv3qZxOJxVat54fOq3ztFr34Y5Dgd3q',
        request_id='9a99879f-7a88-4e68-a4f3-0a7722363304'
    )

    # CSV 파일 읽기
    df = pd.read_csv('뉴스요약_ver2.csv')

    # 키워드를 저장할 리스트
    keys = []

    # Preset text for the API
    preset_text = [{"role": "system", "content": "-문서에서 핵심 키워드를 추출합니다.\n-문서는 증권 뉴스입니다.\n-키워드는 뉴스에서 말하고 있는 주식의 기업명입니다.\n-키워드는 하나만 뽑으세요.\n- 키워드는 한 어절입니다."}]

    # 각 summary 열의 내용을 API로 처리하여 키워드를 추출
    for content in df['summary']:
        preset_text.append({"role": "user", "content": content})
        request_data = {
            'messages': preset_text,
            'topP': 0.6,
            'topK': 0,
            'maxTokens': 300,
            'temperature': 0.2,
            'repeatPenalty': 2.0,
            'stopBefore': [],
            'includeAiFilters': True,
            'seed': 0
        }

        response = completion_executor.execute(request_data)
        keys.append(response.strip())

        # Reset the preset_text for the next iteration
        preset_text.pop()

    # 결과를 새로운 열에 추가
    df['keys'] = keys

    print(df['keys'])


0     id:b01651b1-ef05-41e2-ba71-f76b60b5a25f\nevent...
1     id:bd91cbc7-71a9-4cef-b77f-9551b2e2e61e\nevent...
2     id:69790c4c-ea70-400a-89c5-a13b91121840\nevent...
3     id:4a8d8101-59b2-439e-8301-ddb407144c74\nevent...
4     id:303ee507-0b4a-45f3-8020-2d72cb711c4d\nevent...
                            ...                        
60    id:4470d539-6bd9-434c-969e-9e525029c52c\nevent...
61    id:da38d543-36d6-4f1f-954b-ef518bda4ebf\nevent...
62    id:1a853fd2-26f8-4120-b511-5cf1d8f9d6a8\nevent...
63    id:7b6bca0c-871b-4cfb-bb81-b17307243d2c\nevent...
64    id:fa23093e-bd80-40d7-8141-75673040aa19\nevent...
Name: keys, Length: 65, dtype: object


In [26]:
import json
import re

def extract_keyword(response):
    # Find all JSON objects in the response using regex
    json_objects = re.findall(r'{.*}', response)

    # Extract the "content" field from the relevant JSON object
    for obj in json_objects:
        data = json.loads(obj)
        if "message" in data and "content" in data["message"]:
            content = data["message"]["content"]
            if "키워드 :" in content:
                # Extract the actual keyword
                keyword = content.split("키워드 :")[-1].strip()
                return keyword
    return None

keywords = []
for response in df['keys']:
    keywords.append(extract_keyword(response))
df['keywords'] = keywords
print(df['keywords'])

0          테슬라
1          HMM
2         주식시장
3     매그니피센트 7
4         None
        ...   
60        None
61      LS일렉트릭
62      현대차 그룹
63        None
64        None
Name: keywords, Length: 65, dtype: object


In [27]:
df.to_csv('뉴스기업명추출_ver1.csv', index=False)

결측이 너무 많음...